# 应用优化

In [24]:
import os
import warnings
from IPython.display import Markdown
from setenv import APIKeyManager

# 忽略警告
warnings.filterwarnings('ignore')
# 设置API密钥
key_manager = APIKeyManager().setup_api_key(["DEEPSEEK_API_KEY"])

# !pip install langchain-experimental

# 查看Python版本
!python -V
# 查看安装的库
!pip list | grep -E 'langchain|openai|llm|tiktoken|chromadb|cryptography|duck|unstructured|numpy|scipy'

Python 3.9.21
chromadb                                 0.6.3
cryptography                             44.0.2
duckduckgo_search                        7.5.3
langchain                                0.3.21
langchain-community                      0.3.20
langchain-core                           0.3.47
langchain-deepseek                       0.1.3
langchain-experimental                   0.3.4
langchain-openai                         0.3.9
langchain-text-splitters                 0.3.7
numpy                                    2.0.2
openai                                   1.68.2
scipy                                    1.10.1
tiktoken                                 0.9.0
unstructured                             0.17.2
unstructured-client                      0.31.3


## 1. 性能优化

### 1.1 响应优化

异步和批量处理:

在处理大量请求时，异步和批量处理可以显著提高性能：

1. **异步处理**：使用`async/await`语法实现非阻塞操作，让程序在等待API响应时可以执行其他任务
2. **批量处理**：同时发送多个请求，并行处理多个任务 
下面的代码展示了如何使用LangChain的异步API和回调函数来优化响应性能



In [11]:
from langchain_deepseek import ChatDeepSeek  # 导入DeepSeek聊天模型
import asyncio  # 导入异步IO库，用于实现并发处理

llm = ChatDeepSeek(model="deepseek-chat", temperature=0.7)

In [12]:
# 演示异步生成和批量处理
async def demo_async_and_batch():
    print("===== 单个异步生成示例 =====")
    response = await llm.agenerate(["你好"])
    print(f"生成结果: {response.generations[0][0].text}\n")
    
    print("===== 批量处理示例 =====")
    prompts = ["介绍一下Python", "什么是人工智能", "解释一下大语言模型"]
    # 修改批量处理函数调用
    tasks = [llm.agenerate([prompt]) for prompt in prompts] # 创建异步任务列表
    responses = await asyncio.gather(*tasks) # 并行执行所有任务，提高处理效率
    
    for i, response in enumerate(responses):
        print(f"提示 {i+1} 的结果: {response.generations[0][0].text[:50]}...\n")
    
    return "演示完成"

# 执行演示
await demo_async_and_batch()

===== 单个异步生成示例 =====
生成结果: 你好！😊 很高兴见到你～有什么我可以帮忙的吗？

===== 批量处理示例 =====
提示 1 的结果: Python 是一种高级、解释型、通用的编程语言，由 **Guido van Rossum** 于 ...

提示 2 的结果: 人工智能（Artificial Intelligence，简称 AI）是指由计算机系统模拟、延伸或扩...

提示 3 的结果: 大语言模型（Large Language Model，LLM）是一种基于人工智能的自然语言处理（NL...



'演示完成'

### 1.2 缓存优化

In [20]:
from langchain.globals import set_llm_cache  
from langchain.cache import InMemoryCache  

# 全局设置缓存  
set_llm_cache(InMemoryCache())

In [21]:
# 演示缓存优化效果
import time

# 定义一个函数来测量执行时间
def measure_time(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    return result, end_time - start_time

# 第一次调用（无缓存）
print("===== 缓存优化演示 =====")
prompt = "解释量子计算的基本原理"
result1, time1 = measure_time(llm.invoke, prompt)
print(f"第一次调用（无缓存）:")
print(f"- 耗时: {time1:.2f}秒")
print(f"- 结果前50个字符: {result1.content[:50]}...\n")

# 第二次调用（使用缓存）
result2, time2 = measure_time(llm.invoke, prompt)
print(f"第二次调用（使用缓存）:")
print(f"- 耗时: {time2:.2f}秒")
print(f"- 结果前50个字符: {result2.content[:50]}...\n")

# 显示性能提升
if time1 > 0:
    speedup = time1 / time2 if time2 > 0 else float('inf')
    print(f"性能提升: {speedup:.2f}倍")
    print(f"缓存节省了 {(time1 - time2):.2f} 秒")

===== 缓存优化演示 =====
第一次调用（无缓存）:
- 耗时: 45.26秒
- 结果前50个字符: 量子计算是一种利用量子力学原理（如叠加态和纠缠态）进行信息处理的新型计算范式。其核心原理与传统经典计...

第二次调用（使用缓存）:
- 耗时: 0.00秒
- 结果前50个字符: 量子计算是一种利用量子力学原理（如叠加态和纠缠态）进行信息处理的新型计算范式。其核心原理与传统经典计...

性能提升: 84330.99倍
缓存节省了 45.26 秒


## 2. 用户体验优化 

### 2.1 错误处理

In [39]:
from langchain_core.exceptions import LangChainException 
from langchain_deepseek import ChatDeepSeek

try:
    llm = ChatDeepSeek(model='deepseek-chat')
    response = await llm.ainvoke("Hello")
    print(response.content)
except LangChainException as e:
    # 处理错误
    error_message = f"发生错误：{str(e)}"
    # 返回友好的错误信息
    print(error_message)
except Exception as e:
    print(e)

Hello! 😊 How can I assist you today?


### 2.2 进度反馈

In [44]:
from langchain_deepseek import ChatDeepSeek
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate

# 直接创建回调处理器
callback_handler = StreamingStdOutCallbackHandler()

# 使用回调
llm = ChatDeepSeek(model='deepseek-chat', callbacks=[callback_handler])

# 创建提示模板
prompt = ChatPromptTemplate.from_template("请告诉我关于{topic}的信息")

# 创建链并使用回调
# 在LangChain中，使用|运算符连接时，需要确保两边都是Runnable对象
chain = {"topic": lambda x: x} | prompt | llm

In [47]:
Markdown(chain.invoke("小米").content)

小米（Xiaomi）是一家中国知名的科技公司，成立于2010年，总部位于北京。以下是一些关于小米的关键信息：

### 1. **公司简介**
   - **成立时间**：2010年4月6日。
   - **创始人**：雷军（Lei Jun）。
   - **总部**：中国北京。
   - **业务范围**：智能手机、智能家居、消费电子产品、互联网服务等。

### 2. **主要产品**
   - **智能手机**：小米的智能手机是其核心产品，如小米系列、Redmi系列、POCO系列等。
   - **智能家居**：小米生态链包括智能电视、空气净化器、扫地机器人、智能音箱等。
   - **其他产品**：笔记本电脑、穿戴设备（如手环、手表）、路由器等。

### 3. **商业模式**
   - **高性价比**：小米以“性价比”著称，通过线上销售和高效的供应链管理降低成本。
   - **生态链战略**：投资和支持多家生态链企业，扩展产品线。

### 4. **市场表现**
   - **全球市场**：小米是全球前三的智能手机制造商之一，在印度、欧洲等市场表现强劲。
   - **上市**：2018年7月在香港交易所上市（股票代码：1810.HK）。

### 5. **技术创新**
   - **MIUI**：基于Android的定制操作系统。
   - **研发投入**：在相机技术、快充、AI等领域有显著进展。

### 6. **最新动态**
   - **电动汽车**：2024年推出首款电动汽车SU7，进军汽车行业。
   - **全球化**：持续扩展国际市场，尤其在东南亚、拉美等地。

### 7. **企业文化**
   - **使命**：“让全球每个人都能享受科技带来的美好生活”。
   - **价值观**：用户导向、创新、质量。

如果需要更详细的信息（如财务数据、具体产品参数等），可以进一步提问！